# An advanced look at LLM Agents

Previously, we relied on LangChain to to build the agent. However, this is not necessary, since an agent is nothing more than a fancy while loop. 

**The goal** 

With this notebook you will see what an agent is under the hood, and you can build it based on the LLM output

🌟 So ... let us begin!  

**The steps**

0. [Install dependencies to get the project up and running](#0)
1. [Familiarize yourself with the default wikipedia tool](#1)
        <ol type="a">
        <li>[Explore tool parameters](#1a)</li>
        <li>[Run tool and explore output](#1b)</li>
        </ol>
2. [Build your own tool: weather api](#2)
3. Agent exercises

# Install dependencies <a id='0'></a>

Compile the cell below to install the dependencies. Consider clearing the cell output so it does not clutter your notebook.

In [ ]:
!python3 -m venv venv         
!source venv/bin/activate     
!pip3 install -r helper_functions/requirements.txt

# The key elements

The key elements of the agent remain - we need tools, an LLM that functions as the agent and a prompt with the query. In our example, we'll use ``chatgpt35-turbo`` as the agent LLM, [here](https://platform.openai.com/docs/guides/text-generation/chat-completions-api) you find the documentation on how to query the model through the OpenAI API.

# the tools

In [1]:
from helper_functions.helper_functions import simple_description_formatter
from helper_functions.tools import my_own_wiki_tool, weather_tool
import json
import pprint

# load the tools and format. In plain OpenAI jargon they are called functions.
tools = [my_own_wiki_tool, weather_tool]
function_description = [simple_description_formatter(tool) for tool in tools]
pprint.pprint(function_description[0])

# Store executable functions with their name in dictionary
available_functions = {tool.name: tool for tool in tools}


/Users/maria/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


{'function': {'description': 'wikipedia(query: str) -> str - A wrapper around '
                             'Wikipedia. Useful for when you need to answer '
                             'general questions about people, places, '
                             'companies, facts, historical events, or other '
                             'subjects. Input should be a search query.',
              'name': 'wikipedia',
              'parameters': {'properties': {'query': {'description': 'Input '
                                                                     'search '
                                                                     'query',
                                                      'type': 'string'}},
                             'required': ['query'],
                             'type': 'object'}},
 'type': 'function'}


the prompt is feed as part of the messages object. All input, such as the system prompt, the user question, possibly also chat-history is provided through this object.
Exercise: compile both questions, and compare the answers. Do you see a difference?

The LLM can output two types of answers:
* a string that answers the question, 
* a function-call object, which contains information on which function to call with which arguments.

In [2]:

from helper_functions.keys import client


system_prompt = "You are a friendly, helpful assistant. Your goal is to answer the questions in a concise, but conversational manner."

questions = ["what is the meaning of life?","How many people live in Paris?"]

for question in questions:
  messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

  
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    tools = function_description,
    messages=messages, 
    )

  print(f"Question: {question}")
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Fuction call: {response.choices[0].message.tool_calls}\n")



Question: what is the meaning of life?
Answer: Ah, the age-old question! The meaning of life is a philosophical and existential inquiry that has intrigued humans for centuries. It is a deeply personal and subjective matter, with answers varying greatly depending on individual beliefs, values, and experiences. Some find meaning in relationships, personal growth, helping others, spirituality, or pursuing happiness. Ultimately, the meaning of life is a journey for each person to explore and define for themselves.
Fuction call: None

Question: How many people live in Paris?
Answer: None
Fuction call: [ChatCompletionMessageToolCall(id='call_NpPrg1E5BmhfqGYhJt2C8lUN', function=Function(arguments='{"query":"Population of Paris"}', name='wikipedia'), type='function')]



Here comes the magic: While the LLM requests function calls we 
* store the response object in the messages
* extract per tool the name and the argmemnts of the function to be called
* execute the function
* store the output of the function in the messages object

for more, follow the guide [here](https://platform.openai.com/docs/guides/function-calling)

In [9]:
messages

[{'role': 'system',
  'content': 'You are a friendly, helpful assistant. Your goal is to answer the questions in a concise, but conversational manner.'},
 {'role': 'user', 'content': 'which city is bigger: Paris or Munich?'},
 ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_G2QXZk2u5vTP2Q79VpAspo5R', function=Function(arguments='{"query": "Paris"}', name='wikipedia'), type='function'), ChatCompletionMessageToolCall(id='call_vf5iVlKnhpCfHC5jXDuGwSLs', function=Function(arguments='{"query": "Munich"}', name='wikipedia'), type='function')]),
 {'tool_call_id': 'call_G2QXZk2u5vTP2Q79VpAspo5R',
  'role': 'tool',
  'name': 'wikipedia',
  'content': "Page: Paris\nSummary: Paris is the capital and largest city of France. With an official estimated population of 2,102,650 residents as of 1 January 2023 in an area of more than 105 km2 (41 sq mi), Paris is the fourth-largest city in the European Union and the 30th most de

In [24]:
# question = "How many people live in Paris?"
# question = "which city is bigger: Paris or Munich?"
question = 'How is the weather where the king of the netherlands lives?'

messages = [
      {"role": "system", "content": system_prompt},
      {"role": "user", "content": question}
    ]

response = client.chat.completions.create(
  model="gpt-3.5-turbo",
  tools = function_description,
  messages=messages, 
  )

print('==== Initial LLM response ====')
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

# while the response requests function calls
while response.choices[0].message.tool_calls:
    
  # store response message with all function calls
  response_message = response.choices[0].message
  messages.append(response_message)

  # execute each tool individually
  for tool_call in response.choices[0].message.tool_calls:
    print('==== Function call ====')

    # function name and arguments
    function_name = tool_call.function.name
    function_args = json.loads(tool_call.function.arguments)
    print(f'Calling function "{function_name}" with arguments {function_args}.')

    # execute function call 
    function_response = available_functions[function_name].invoke(function_args)
    print(f'Function call response:\n{function_response}\n')

    # append function response to messages
    messages.append({
        "tool_call_id":tool_call.id, 
        "role": "tool", 
        "name": function_name, 
        "content": function_response
    })
    
  # get a new response from LLM
  response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    tools = function_description,
    messages=messages, 
  )

  print('== Intermediate LLM response ==')
  print(f"Answer: {response.choices[0].message.content}")
  print(f"Function call: {response.choices[0].message.tool_calls}\n")

print('==== Final LLM response ====')
print("Question: ", question)
print(f"Answer: {response.choices[0].message.content}")
print(f"Function call: {response.choices[0].message.tool_calls}\n")

==== Initial LLM response ====
Answer: None
Function call: [ChatCompletionMessageToolCall(id='call_CVZj8JoHIpJn9H4ltCOXhDZB', function=Function(arguments='{"city":"The Hague"}', name='weather'), type='function')]

==== Function call ====
Calling function "weather" with arguments {'city': 'The Hague'}.
Function call response:
Current temperature in The Hague: 14.1°C

== Intermediate LLM response ==
Answer: The current temperature in The Hague, where the King of the Netherlands lives, is 14.1°C.
Function call: None

==== Final LLM response ====
Question:  How is the weather where the king of the netherlands lives?
Answer: The current temperature in The Hague, where the King of the Netherlands lives, is 14.1°C.
Function call: None

